# **Environment Setup**

In [63]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.14 s (started: 2022-09-02 12:35:15 +00:00)


In [64]:
!pip install tensorflow_addons
!pip install iterative-stratification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 6.37 s (started: 2022-09-02 12:35:18 +00:00)


In [65]:
from zipfile import ZipFile
import pandas as pd
import spacy
from string import punctuation
from bs4 import BeautifulSoup
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa

time: 83.9 ms (started: 2022-09-02 12:35:24 +00:00)


In [66]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
time: 16.3 s (started: 2022-09-02 12:35:24 +00:00)


# **Import from Kaggle**

In [67]:
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
time: 1.47 s (started: 2022-09-02 12:35:41 +00:00)


In [68]:
!kaggle datasets download stackoverflow/stacksample

stacksample.zip: Skipping, found more recently modified local copy (use --force to force download)
time: 1.6 s (started: 2022-09-02 12:35:42 +00:00)


In [69]:
with ZipFile("/content/stacksample.zip","r") as zip_ref:
  zip_ref.extractall("/content/dataset")

time: 41.1 s (started: 2022-09-02 12:35:44 +00:00)


# **Data Exploration and Cleaning**

In [70]:
questions = pd.read_csv("/content/dataset/Questions.csv",encoding='utf-8',encoding_errors='replace')

time: 32 s (started: 2022-09-02 12:36:25 +00:00)


In [71]:
tags = pd.read_csv("/content/dataset/Tags.csv")

time: 1.47 s (started: 2022-09-02 12:36:57 +00:00)


In [8]:
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


time: 27.9 ms (started: 2022-09-02 11:14:13 +00:00)


In [9]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


time: 7.99 ms (started: 2022-09-02 11:14:13 +00:00)


In [10]:
questions = questions.drop(labels=["OwnerUserId","CreationDate","ClosedDate"],axis=1)

time: 94.6 ms (started: 2022-09-02 11:14:13 +00:00)


In [11]:
def add_title_to_question(title,body):
  question = title+"\n"+body
  return question

questions["Title"] = questions.apply(lambda row: add_title_to_question(row["Title"],row["Body"]),axis=1)

time: 17.6 s (started: 2022-09-02 11:14:13 +00:00)


In [12]:
questions = questions[["Id","Title","Score"]]

time: 42.1 ms (started: 2022-09-02 11:14:31 +00:00)


In [13]:
questions["Question"] = questions["Title"]
questions = questions[["Id","Question","Score"]]

time: 304 ms (started: 2022-09-02 11:14:31 +00:00)


In [72]:
tags = tags.dropna()

time: 401 ms (started: 2022-09-02 12:36:58 +00:00)


In [15]:
tags[tags['Tag'].apply(lambda x: not isinstance(x, str))]

,Id,Tag


time: 779 ms (started: 2022-09-02 11:14:32 +00:00)


In [73]:
tags = tags.groupby(["Id"])["Tag"].apply(tuple)

time: 18.6 s (started: 2022-09-02 12:36:59 +00:00)


In [17]:
tags.head()

Id
80                           (flex, actionscript-3, air)
90     (svn, tortoisesvn, branch, branching-and-merging)
120                              (sql, asp.net, sitemap)
180    (algorithm, language-agnostic, colors, color-s...
260         (c#, .net, scripting, compiler-construction)
Name: Tag, dtype: object

time: 8.82 ms (started: 2022-09-02 11:14:49 +00:00)


In [18]:
df = pd.merge(tags,questions,left_on="Id",right_on="Id")

time: 707 ms (started: 2022-09-02 11:14:50 +00:00)


In [19]:
df = df[["Question","Tag","Score"]]

time: 218 ms (started: 2022-09-02 11:14:50 +00:00)


In [20]:
df.head()

,Question,Tag,Score
0,SQLStatement.execute() - multiple queries in o...,"(flex, actionscript-3, air)",26
1,Good branching and merging tutorials for Torto...,"(svn, tortoisesvn, branch, branching-and-merging)",144
2,ASP.NET Site Maps\n<p>Has anyone got experienc...,"(sql, asp.net, sitemap)",21
3,Function for creating color wheels\n<p>This is...,"(algorithm, language-agnostic, colors, color-s...",53
4,Adding scripting functionality to .NET applica...,"(c#, .net, scripting, compiler-construction)",49


time: 11.1 ms (started: 2022-09-02 11:14:50 +00:00)


The dataset is way too big to be processed in a short amount of time here, so we will use the 5% of it and eventually increase this percentage if the results are not good enough.
We will filter the rows by the score and sort by the number of occurrences of the tags

In [21]:
df = df[df['Score'] > 5]

time: 24 ms (started: 2022-09-02 11:14:50 +00:00)


In [22]:
df1 = df['Tag'].value_counts().sort_values(ascending=False).to_frame()
df1.columns = ['Count']
df1['Tag'] = df1.index

time: 106 ms (started: 2022-09-02 11:14:51 +00:00)


In [23]:
df1.head()

,Count,Tag
"(android,)",437,"(android,)"
"(javascript,)",303,"(javascript,)"
"(python,)",297,"(python,)"
"(java,)",275,"(java,)"
"(javascript, jquery)",242,"(javascript, jquery)"


time: 14.1 ms (started: 2022-09-02 11:14:51 +00:00)


In [24]:
df = pd.merge(df1,df,on="Tag")

time: 76.8 ms (started: 2022-09-02 11:14:51 +00:00)


In [25]:
df = df[['Tag','Question']]

time: 33.3 ms (started: 2022-09-02 11:14:51 +00:00)


In [26]:
df.head()

,Tag,Question
0,"(android,)",Android - How Do I Set A Preference In Code\n<...
1,"(android,)",Lock android app after a certain amount of idl...
2,"(android,)",What are Android application update issues and...
3,"(android,)",How to add an extra language input to Android?...
4,"(android,)",How to create our own Listener interface in an...


time: 10.3 ms (started: 2022-09-02 11:14:51 +00:00)


In [27]:
df = df.loc[:12642]

time: 7.89 ms (started: 2022-09-02 11:14:51 +00:00)


# **Data Preprocessing**

In [32]:
nlp = spacy.load("en_core_web_sm")

time: 721 ms (started: 2022-09-02 11:15:20 +00:00)


In [33]:
def is_ok(token):
  if len(token) < 3:
    return False
  if token.is_punct:
    return False
  if token.is_stop:
    return False
  if token.is_space:
    return False

  return True
  
def preprocessing(text):
  text_without_tags = BeautifulSoup(text).get_text()
  for char in text_without_tags:
    if char.isdigit() or char in punctuation:
      text_without_tags = text_without_tags.replace(char," ")

  doc = nlp(text_without_tags)
  return [token.lemma_.lower() for token in doc if is_ok(token)]
  

time: 3.62 ms (started: 2022-09-02 11:15:21 +00:00)


In [34]:
print(preprocessing(df['Question'][1])) #let's try the preprocessing pipeline

['lock', 'android', 'app', 'certain', 'idle', 'time', 'android', 'application', 'require', 'password', 'enter', 'activity', 'want', 'able', 'automatically', 'send', 'application', 'password', 'entry', 'screen', 'application', 'idle', 'fix', 'time', 'application', 'multiple', 'activity', 'like', 'timeout', 'global', 'activity', 'wouldn', 'sufficient', 'create', 'timer', 'thread', 'onpause', 'method', 'activity', 'sure', 'good', 'definition', 'application', 'idle', 'activity', 'active', 'sufficient']
time: 66.4 ms (started: 2022-09-02 11:15:21 +00:00)


In [35]:
df['Question'] = [preprocessing(text) for text in df['Question']]

time: 5min 28s (started: 2022-09-02 11:15:21 +00:00)


In [36]:
df.head()

,Tag,Question
0,"(android,)","[android, set, preference, code, android, appl..."
1,"(android,)","[lock, android, app, certain, idle, time, andr..."
2,"(android,)","[android, application, update, issue, good, pr..."
3,"(android,)","[add, extra, language, input, android, possibl..."
4,"(android,)","[create, listener, interface, android, help, c..."


time: 12.8 ms (started: 2022-09-02 11:20:49 +00:00)


In [38]:
mlb = MultiLabelBinarizer()
tags = mlb.fit_transform(df['Tag'])
onehot_tags = pd.DataFrame(tags,columns=mlb.classes_)

time: 157 ms (started: 2022-09-02 11:20:52 +00:00)


In [39]:
df_final = pd.merge(df,onehot_tags,left_index=True,right_index=True)

time: 117 ms (started: 2022-09-02 11:20:52 +00:00)


In [40]:
df_final.head()

,Tag,Question,.net,actionbarsherlock,activerecord,adb,adt,ajax,algorithm,android,...,xaml,xcode,xcode4,xml,xml-serialization,xpath,xslt,yii,zend-framework,zend-framework2
0,"(android,)","[android, set, preference, code, android, appl...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"(android,)","[lock, android, app, certain, idle, time, andr...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,"(android,)","[android, application, update, issue, good, pr...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,"(android,)","[add, extra, language, input, android, possibl...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"(android,)","[create, listener, interface, android, help, c...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


time: 20.4 ms (started: 2022-09-02 11:20:53 +00:00)


In [41]:
df_x = df_final['Question']
df_y = df_final.drop(['Tag','Question'],axis=1)

time: 48.9 ms (started: 2022-09-02 11:20:53 +00:00)


In [43]:
y = df_y.to_numpy()
x = df_x.to_numpy()

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(x, y):
  train_x, test_x = x[train_index], x[test_index]
  train_y, test_y = y[train_index], y[test_index]

time: 2.84 s (started: 2022-09-02 11:20:58 +00:00)


In [44]:
def dummy(doc):
  return doc


tfidf_vectorizer = TfidfVectorizer(
    preprocessor=dummy,
    tokenizer=dummy,
    analyzer="word",
    token_pattern=None
)

train_x = tfidf_vectorizer.fit_transform(train_x)
test_x = tfidf_vectorizer.transform(test_x)

time: 494 ms (started: 2022-09-02 11:21:00 +00:00)


In [45]:
train_y = train_y.astype('int')
test_y = test_y.astype('int')

time: 26.2 ms (started: 2022-09-02 11:21:01 +00:00)


# **Training and Testing with Scikit**

**Logistic Regression**

In [163]:
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(train_x,train_y)
predictions = clf.predict(test_x)

time: 2min 20s (started: 2022-09-01 16:13:35 +00:00)


In [164]:
print("Accuracy:",accuracy_score(test_y,predictions))
print("Precision:",precision_score(test_y,predictions,average="micro"))
print("Recall:",recall_score(test_y,predictions,average="micro"))
print("F1 Score:",f1_score(test_y,predictions,average="micro"))

Accuracy: 0.15023847376788554
Precision: 0.8844086021505376
Recall: 0.23725961538461537
F1 Score: 0.3741470811220621
time: 506 ms (started: 2022-09-01 16:16:02 +00:00)


**Support Vector Machine**

In [95]:
clf = OneVsRestClassifier(SVC())
clf.fit(train_x,train_y)
predictions = clf.predict(test_x)

Accuracy: 0.21899841017488075
F1 Score: 0.4839106734657992
time: 19min 56s (started: 2022-09-01 13:22:29 +00:00)


In [102]:
print("Accuracy:",accuracy_score(test_y,predictions))
print("Precision:",precision_score(test_y,predictions,average="micro"))
print("Recall:",recall_score(test_y,predictions,average="micro"))
print("F1 Score:",f1_score(test_y,predictions,average="micro"))

Accuracy: 0.21899841017488075
Precision: 0.9011132940406025
Recall: 0.33076923076923076
F1 Score: 0.4839106734657992
time: 445 ms (started: 2022-09-01 13:47:44 +00:00)


**XGBoost**

Scikit-learn XGBoost wrapper is too slow to be computed on Colab Free provided hardware on this problem

In [ ]:
clf = OneVsRestClassifier(XGBClassifier())
clf.fit(train_x,train_y)
predictions = clf.predict(test_x)

In [ ]:
print("Accuracy:",accuracy_score(test_y,predictions))
print("Precision:",precision_score(test_y,predictions,average="micro"))
print("Recall:",recall_score(test_y,predictions,average="micro"))
print("F1 Score:",f1_score(test_y,predictions,average="micro"))

# **Training and Testing with NN**

In [47]:
train_x = tf.expand_dims(train_x.toarray(), axis=-1)
test_x = tf.expand_dims(test_x.toarray(),axis=-1)

time: 3.13 s (started: 2022-09-02 10:15:24 +00:00)


Since Colab free only provides 12 GB of RAM, we save the train and test sets on Colab drive as np arrays, we restart the session releasing all the occupied RAM and then we load the arrays again to fit the model.

In [48]:
np.save("/content/train_x",train_x)
np.save("/content/train_y",train_y)
np.save("/content/test_x",test_x)
np.save("/content/test_y",test_y)

time: 15.2 s (started: 2022-09-02 10:15:27 +00:00)


Between these two snippets, we restart the session

In [2]:
train_x = np.load("/content/train_x.npy")
train_y = np.load("/content/train_y.npy")
test_x = np.load("/content/test_x.npy")
test_y = np.load("/content/test_y.npy")

**Artificial Neural Network**

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[33539,]))
model.add(tf.keras.layers.Dense(256,activation="relu"))
model.add(tf.keras.layers.Dense(256,activation="relu"))
model.add(tf.keras.layers.Dense(256,activation="relu"))
model.add(tf.keras.layers.Dense(430,activation="sigmoid"))

In [45]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy",tf.metrics.Precision(),tf.metrics.Recall(),tfa.metrics.F1Score(num_classes=430,average='micro')])

In [44]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True,verbose=1,monitor='val_accuracy')

In [46]:
model.fit(train_x,train_y,epochs=30,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/30
254/254 [==============================] - 4s 12ms/step - loss: 5.1700e-05 - accuracy: 0.7764 - precision_3: 0.9980 - recall_3: 0.9976 - f1_score: 0.7944 - val_loss: 0.1041 - val_accuracy: 0.2838 - val_precision_3: 0.5658 - val_recall_3: 0.2784 - val_f1_score: 0.3277
Epoch 2/30
254/254 [==============================] - 2s 9ms/step - loss: 6.9044e-05 - accuracy: 0.7869 - precision_3: 0.9971 - recall_3: 0.9971 - f1_score: 0.7959 - val_loss: 0.0999 - val_accuracy: 0.2937 - val_precision_3: 0.5493 - val_recall_3: 0.2906 - val_f1_score: 0.3310
Epoch 3/30
254/254 [==============================] - 2s 9ms/step - loss: 9.9858e-05 - accuracy: 0.7883 - precision_3: 0.9962 - recall_3: 0.9954 - f1_score: 0.7980 - val_loss: 0.0975 - val_accuracy: 0.2779 - val_precision_3: 0.5253 - val_recall_3: 0.2915 - val_f1_score: 0.3226
Epoch 4/30
254/254 [==============================] - 2s 9ms/step - loss: 2.0406e-04 - accuracy: 0.7790 - precision_3: 0.9924 - recall_3: 0.9916 - f1_score: 0.7929 -

In [47]:
model.evaluate(test_x,test_y)

79/79 [==============================] - 1s 8ms/step - loss: 0.0466 - accuracy: 0.4801 - precision_3: 0.6350 - recall_3: 0.4976 - f1_score: 0.4917


[0.04655624181032181,
 0.4801271855831146,
 0.6349693536758423,
 0.4975961446762085,
 0.4917447566986084]